In [8]:
# Import necessary packages
import os 
import earthpy as et

import folium
from folium import plugins

import rasterio as rio
from rasterio.warp import calculate_default_transform, reproject, Resampling



In [9]:
# Create a map using the Map() function and the coordinates for Boulder, CO
m = folium.Map(location=[40.0150, -105.2705])

# Display the map
m


In [10]:
# Create a map using Stamen Terrain as the basemap
m = folium.Map(location=[40.0150, -105.2705],
              tiles = 'Stamen Terrain')

# Display map
m

In [11]:
# Create a map using Stamen Terrain, centered on Boulder, CO
m = folium.Map(location=[40.0150, -105.2705], 
              tiles = 'Stamen Terrain')

# Add marker for Boulder, CO
folium.Marker(
    location=[40.009515, -105.242714], # coordinates for the marker (Earth Lab at CU Boulder)
    popup='Earth Lab at CU Boulder', # pop-up label for the marker
    icon=folium.Icon()
).add_to(m)

# Display m
m


In [12]:
# Create variables for destination coordinate system and the name of the projected raster
dst_crs = 'EPSG:4326' 
out_path = '/Users/habib13inch/Documents/13/code/js/geotiff.js/python/colorado-flood/spatial/boulder-leehill-rd/outputs/reproj_post_DTM.tif'


# Use rasterio package as rio to open and project the raster
with rio.open('/Users/habib13inch/Documents/13/code/js/geotiff.js/python/colorado-flood/spatial/boulder-leehill-rd/post-flood/lidar/post_DTM.tif') as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })
 

    # Use rasterio package as rio to write out the new projected raster
    # Code uses loop to account for multi-band rasters
    with rio.open(out_path, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
            source=rio.band(src, i),
            destination=rio.band(dst, i),
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=transform,
            dst_crs=dst_crs,
            resampling=Resampling.nearest)    
# Use rasterio to import the reprojected data as img
with rio.open(out_path) as src:
    boundary = src.bounds
    img = src.read()
    nodata = src.nodata

In [15]:
# Create a map using Stamen Terrain, centered on study area with set zoom level
m = folium.Map(location=[40.06, -105.30],
                   tiles='Stamen Terrain', zoom_start = 13)

# Overlay raster called img using add_child() function (opacity and bounding box set)
m.add_child(folium.raster_layers.ImageOverlay(img[0], opacity=.7, 
                                 bounds =[[40.05577828237005, -105.32837712340124], [40.073923431943214, -105.28139535136515]]))

# Display map
m

In [16]:
m